In [97]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input
import numpy as np
import pandas as pd


In [98]:
data_test = pd.read_csv("df_test.csv")
data_training = pd.read_csv("df_training.csv")

In [99]:
x_train=data_training["x"] # El entrenamiendo del texto
y_train=data_training["y"] # Las etiquetas del entrenamiento


x_test=data_test["x"] # La prueba del texto
y_test=data_test["y"] # Las etiquetas para las pruebas



In [100]:
# Crear el vectorizador
vectorizer = TfidfVectorizer(max_features=1000)  # Limitar el número de características

# Ajustar y transformar los datos de entrenamiento
X_train_vect = vectorizer.fit_transform(x_train)

# Transformar los datos de prueba
X_test_vect = vectorizer.transform(x_test)

X_train_vect = X_train_vect.toarray()  
X_train_vect = X_train_vect.astype('float32') 



In [101]:
# Crear el codificador
label_encoder = LabelEncoder()

# Ajustar y transformar las etiquetas de entrenamiento y prueba
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Verifica las primeras etiquetas codificadas
print(y_train[:10])  # Muestra las primeras 10 etiquetas codificadas

[5 5 1 4 1 5 0 2 3 4]


In [102]:
#Creación del modelo

modelo = Sequential([
    Input(shape=(X_train_vect.shape[1],)),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y_train)), activation='softmax')
])

modelo.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

# Entrenar el modelo
modelo.fit(X_train_vect, y_train, epochs=5, batch_size=32)



Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4303 - loss: 1.4653
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8370 - loss: 0.4945
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8775 - loss: 0.3614
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8942 - loss: 0.3054
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9112 - loss: 0.2513


In [103]:
# Evaluar el modelo
test_loss, test_acc = modelo.evaluate(X_test_vect.toarray(), y_test)
print(f"Test accuracy: {test_acc}")


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5785 - loss: 2.6356
Test accuracy: 0.5550000071525574


In [104]:
# Hacer predicciones
predicciones = modelo.predict(X_test_vect)

# Convertir las predicciones a etiquetas
predicciones_clase = predicciones.argmax(axis=1)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
